In [ ]:
import numpy as np #importing numpy
import pandas as pd #importing pandas

In [ ]:
movies = pd.read_csv('tmdb_5000_movies.csv') #read the movies file
credits = pd.read_csv('tmdb_5000_credits.csv') #read the credits file

In [ ]:
movies.head(1) #to show about one movie

In [ ]:
movies.merge(credits,on='title') #command use to merge

In [ ]:
movies.merge(credits,on='title').shape #to know the shape of the dataset

In [ ]:
movies.shape #to know the movies shape
#credits.shape #to know the credit shape

In [ ]:
movies = movies.merge(credits,on='title') #adding the caste and crew datas form credits dataset to the movies

In [ ]:
movies.head(1) 

In [ ]:
#to select the number of needed columns and remove the other columns
#genres
#id
#keywords
#title
#overview
#cast
#crew

movies =  movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [ ]:
movies.info() #to show all the columns

In [ ]:
movies.head()

In [ ]:
movies.isnull().sum() #to know if there is any missing data or not,if then we have to remove them

In [ ]:
movies.dropna(inplace=True) #since there is 3 overview values so drop them using dropna function

In [ ]:
movies.duplicated().sum() #to know if there is any duplicate data or not,if then we have to remove them

In [ ]:
movies.iloc[0].genres

In [ ]:
# i have to make the genres as ['Action','Adventure','Fantasy','name','Science Fiction']
import ast

In [ ]:
def convert(obj): #its a helper function to sort out the above
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [ ]:
movies['genres'] = movies['genres'].apply(convert)

In [ ]:
movies.head()

In [ ]:
 movies['keywords'] = movies['keywords'].apply(convert)

In [ ]:
movies.head()

In [ ]:
# to convert cast column in dictionary
def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L

In [ ]:
movies['cast'] = movies['cast'].apply(convert3) #for each movie there will be three outcomes

In [ ]:
movies.head()

In [ ]:
#for crew column
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

In [ ]:
movies['crew']=movies['crew'].apply(fetch_director)

In [ ]:
movies.head()

In [ ]:
movies['overview']=movies['overview'].apply(lambda x:x.split()) #for creating the overview as a list

In [ ]:
movies.head()

In [ ]:
#we have to revome the space in between example as (samworthington=sam worthington)
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies.head()

In [ ]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [ ]:
movies.head()

In [ ]:
new_df = movies[['movie_id','title','tags']] #creating a new df,where we keep movie_id, 'title' and tags column

In [ ]:
#to convert the list of tags to string
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
import nltk
nltk.download() #importing nltk(natural language toolkit)

In [ ]:
from nltk.stem.porter import PorterStemmer #we will use stemming
ps = PorterStemmer() #if we use ps.stem('loved or loving') it will give output as love

In [ ]:
#Helper function
def stem(text):
    y = []
    
    for i in text.split():
        y.append(ps.stem(i))
        
    return " ".join(y)

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem) #that is it will apply stemming in the whole 'tags'

In [ ]:
new_df.head() #for all the column at we have converted into strings

In [ ]:
#for a particular column
new_df['tags'][0]

In [ ]:
#To convert all the string into lowercase
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

In [ ]:
new_df.head()

In [ ]:
#to remove the stopwords and making all the words in the inform of a vector
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
#converting explicitly into a numpy array
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
#Now all the movies are in the vector form
vectors

In [ ]:
vectors[0]

In [ ]:
#all the words and digits
cv.get_feature_names()

In [ ]:
#since there is so many similar types of words, so we have to remove those,example-actor,actors
#now we will use stemming to remove the similar types of words
#showing only the root word such as marine = marin 
stem('In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron')

In [ ]:
#now we will use cosine similarity between one movie with another movie
#importing cosine_similarity
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
#passing the whole vectors
similarity = cosine_similarity(vectors)

In [ ]:
#we will sort the list in ascending order will only first five values,but sorted based on the 2nd value
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]

In [ ]:
#now we will make a function,if i chose a movie then it will recomend 5 similar types of movies
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    
    for i in movies_list:
        print(new_df.iloc[i[0]].title) # it will simply show the title and the closest similar title

In [ ]:
#the output will be listed in the following way
recommend('Batman')